In [ ]:
#!pip install tweet-preprocessor

In [10]:
import preprocessor as p

In [137]:
import pandas as pd
import json
import re

In [105]:
pwd

'/home/jovyan/capstone-34/34.210.104.115/eg_twitter_pickled'

In [106]:
pd.options.display.max_colwidth = 75

In [107]:
#cd ..

In [108]:
cd ../eg_twitter_raw/eg_timelines_raw/

/home/jovyan/capstone-34/34.210.104.115/eg_twitter_raw/eg_timelines_raw


In [109]:
ls

user_timeline_mamhosni.jsonl


### Turn query documents (tweets) into generators

In [22]:
def load_tweets(file):
    with open(file, 'r') as f:
        tweets = (json.loads(line) for line in f.readlines())
    return tweets

In [110]:
query_gen = load_tweets('user_timeline_mamhosni.jsonl')

### Turn query generators into dataframes

In [111]:
def tweet_to_df(tweets):
    data = {'text': [], 'screen_name': [], 'geo': [],
       }
    
    for t in tweets:
    
        data['text'].append(t['text'])
        data['screen_name'].append(t['user']['screen_name'])
        data['geo'].append(t['user']['location'])
    
    return pd.DataFrame(data)

In [112]:
query_df = tweet_to_df(query_gen)

In [113]:
query_df.sample(4)

,geo,screen_name,text
684,الخُن,mamhosni,RT @galalaldeen713: @mamhosni مش أنا 😂😂😂 https://t.co/XwBwdZ40co
251,الخُن,mamhosni,فصبرٌ جميل والله المستعان على ما تصفون 🙏🏼🙏🏼🙏🏼 https://t.co/adnEcour36
2982,الخُن,mamhosni,RT @Thawri2011: @mamhosni مشيها https://t.co/KZsTmIVZvE
1834,الخُن,mamhosni,RT @adel_Esa: سيأتي يوم ما .. وحاكم ما .. فى عصر ما .. سيجمع مشايخ وكهن...


### Clean tweets (removes emojis, links, special characters, and ASCII chars/digits from tweet text)

Using a regular expression such as `[0-9A-Za-z:!/._?؟'+-=—]` can filter out much of the unwanted data and retain Arabic characters but still seems to also retain a more recently released generation of emojis. Experiment with different regex variations. Update tweet-preprocessor to version 0.5.0 or later. 


In [114]:
def cleaner(message):
    message = re.sub("([0-9A-Za-z:&!/._?)؟'+-=—])", " ", message)
    return message


In [115]:
cleaned_text = []

for tweet in query_df['text']:
    cleaned_text.append(p.clean(cleaner(tweet)))

query_df['cleaned_text']=cleaned_text

In [116]:
cleaned_name = []

for tweet in query_df['screen_name']:
    cleaned_name.append(p.clean(tweet))

query_df['cleaned_name']=cleaned_name

In [117]:
cleaned_geo = []

for tweet in query_df.geo:
    cleaned_geo.append(p.clean(str(tweet)))

query_df['cleaned_geo']=cleaned_geo


In [118]:
query_df.sample(5)

,geo,screen_name,text,cleaned_text,cleaned_name,cleaned_geo
1842,الخُن,mamhosni,@77_Rania قلنا مليون مرة الخرفان عايزه برسيم بس ليه كدا بس \nربنا ينفخه...,قلنا مليون مرة الخرفان عايزه برسيم بس ليه كدا بس ربنا ينفخهم بحق جاه النبي,mamhosni,الخُن
2554,الخُن,mamhosni,RT @yasmeen230479: @mamhosni 😔😔😔😔😔😔,,mamhosni,الخُن
224,الخُن,mamhosni,@sokkarnabat90 يعني عشان ناس عرفناهم من ٣٠ سنة خلاص بقوا أصحابنا 🤔😂😂😂😂,يعني عشان ناس عرفناهم من سنة خلاص بقوا أصحابنا 🤔,mamhosni,الخُن
813,الخُن,mamhosni,#اسباب_السعاده_والانبساط \n🙌😌😌😌 https://t.co/9V1BplX8RX,السعاده والانبساط,mamhosni,الخُن
1010,الخُن,mamhosni,RT @narmr22: @mamhosni اي نعم زغلوله 😂😂😂😂😂😂😂,اي نعم زغلوله,mamhosni,الخُن


In [119]:
query_df.shape

(3184, 6)

### Remove duplicated tweets

In [120]:
query_df['cleaned_text'].duplicated().sample(5)

394     False
1759    False
1534    False
435     False
1063    False
Name: cleaned_text, dtype: bool

In [121]:
query_cleaned = query_df.drop_duplicates(['cleaned_text'], keep=False)
query_cleaned = query_cleaned.drop(['screen_name', 'text', 'geo'], axis=1)

In [122]:
query_cleaned.sample(4)

,cleaned_text,cleaned_name,cleaned_geo
1940,كتير فعلا بلاقي التويته محفوظة معرفش ليه النت وعمايله بقا,mamhosni,الخُن
2035,كلنا مستهدفين,mamhosni,الخُن
815,ها ايه,mamhosni,الخُن
1432,العباسية,mamhosni,الخُن


In [123]:
query_cleaned.fillna('None')

,cleaned_text,cleaned_name,cleaned_geo
0,دي كانت لعبة بس بصيد بيها حسابات الدول الأجنبية بالعربي هههههههه,mamhosni,الخُن
1,وبعدين بقا ارسوا على بر اليابان ولا السويد,mamhosni,الخُن
2,تصبحون على وطن خالي من الدسم,mamhosni,الخُن
3,ورايا برد لا لو سمحت انا رجعت فرولايه,mamhosni,الخُن
4,حبيبي وابن حبيبي والنييييعمه ياحوووب,mamhosni,الخُن
5,دي مش اي مصلحه دا سحلب بالمخسرات يا حجوج,mamhosni,الخُن
6,كده هنخسر بعض انا لسه مقاطعه امي بسبب المواعين,mamhosni,الخُن
7,يالا يا مستكة نامي وراكي برد بدري,mamhosni,الخُن
8,مستكة جايه ف مصلحة,mamhosni,الخُن
9,منا رئيس بردو يا كبير المهندسين بس رئيس الخُن,mamhosni,الخُن


In [124]:
query_cleaned.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2867 entries, 0 to 3183
Data columns (total 3 columns):
cleaned_text    2867 non-null object
cleaned_name    2867 non-null object
cleaned_geo     2867 non-null object
dtypes: object(3)
memory usage: 89.6+ KB


### Consolidate different geo tags

In [125]:
locations = {'Saudi Arabia': ['Kingdom of Saudi Arabia', 'MAKKAH', 'saudi arabia', 'jeddah', 'مكة المكرمة', 'المملكة العربية', 'Jeddah', 'Riyadh', 'المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'jeddah , saudi arabia', 'الرياض', 'آلقصيم ، بريده', 'جدة, المملكة العربية السعودية', 'المدينة المنورة, المملكة العربية السعودية', 'Riyadh, Kingdom of Saudi Arabia', 'makkah - saudi arabia', 'جده', 'Dammam', 'Dammam, Eastern', 'Al Khobar, Kingdom of Saudi Arabia'],
             'Kuwait': ['الكويت', 'Alkuwait', 'kuwait', 'KUWAIT', 'Kuwait , Salwa', 'Qortuba, Kuwait', 'دولة الكويت', 'العاصمه, دولة الكويت', 'The Capital, Kuwait', 'Al Salam, Kuwait', 'Salwa, Kuwait', 'كوكب الشرق - Kuwait'],
             'Egypt': ['Domiat, Egypt', 'Damitta', 'Alexandria, Egypt', 'Damitta-Egypt', 'Giza, Egypt', 'New damieta city', 'مصر', 'القاهرة, مصر', 'El Behera, Egypt', 'الجيزة, مصر', 'Elmansoura', 'El Menia, Egypt', 'El Qaliobia, Egypt', 'البحيرة, مصر', 'Domyat elgadeda', 'El Sharkia, Egypt', 'sharkia_fakous', 'Cairo, Egypt', 'Alexandria,Egypt', 'ELsharkia , Egypt', 'شمال سيناء, مصر', 'كفر الشيخ, مصر', 'Port Said, Egypt', 'Beni Suef, Egypt', 'القليوبية, مصر', 'بنها', 'Aswan', 'Beni Suef, Egypt', 'Zagazig,Egypt', 'Ismalia, Egypt', 'Kafr El Shikh, Egypt', 'القاهرة, مصر', 'البحيرة, مصر', 'live in q8 .. bas masrya', 'القاهرة, مصر', 'Domiat, Egypt', 'Alexandria, Egypt', 'Cairo, Egypt', 'El Menia', 'أسكندرية', 'Alexandria', 'tanta', 'giza,egypt', 'Giza Governorate, Egypt', 'Mansoura', 'Mansura,Egypt','المنوفية, مصر', 'مصر الاسكندرية','El Sharkia, Egypt', 'El Daqahlia, Egypt', 'Cairo -Egypt', 'Cairo , Egypt', 'Aswan, Egypt', 'Mansurah, Egypt', 'الأسكندرية, مصر', 'alexandria', 'الأقصر, مصر', 'Cairo', 'El Gharbia, Egypt', 'egypt', 'Mansoura,Egypt', 'Beni Suef, Egypt', 'Mansoura', 'El Daqahlia, Egypt', 'El Monofia, Egypt']
            }

In [126]:
'MAKKAH' in list(locations.values())[0]

True

In [127]:
'MAKKAH' in list(locations.values())[1]

False

In [128]:
locations.values()

dict_values([['Kingdom of Saudi Arabia', 'MAKKAH', 'saudi arabia', 'jeddah', 'مكة المكرمة', 'المملكة العربية', 'Jeddah', 'Riyadh', 'المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'jeddah , saudi arabia', 'الرياض', 'آلقصيم ، بريده', 'جدة, المملكة العربية السعودية', 'المدينة المنورة, المملكة العربية السعودية', 'Riyadh, Kingdom of Saudi Arabia', 'makkah - saudi arabia', 'جده', 'Dammam', 'Dammam, Eastern', 'Al Khobar, Kingdom of Saudi Arabia'], ['الكويت', 'Alkuwait', 'kuwait', 'KUWAIT', 'Kuwait , Salwa', 'Qortuba, Kuwait', 'دولة الكويت', 'العاصمه, دولة الكويت', 'The Capital, Kuwait', 'Al Salam, Kuwait', 'Salwa, Kuwait', 'كوكب الشرق - Kuwait'], ['Domiat, Egypt', 'Damitta', 'Alexandria, Egypt', 'Damitta-Egypt', 'Giza, Egypt', 'New damieta city', 'مصر', 'القاهرة, مصر', 'El Behera, Egypt', 'الجيزة, مصر', 'Elmansoura', 'El Menia, Egypt', 'El Qaliobia, Egypt', 'البحيرة, مصر', 'Domyat elgadeda', 'El Sharkia, Egypt', 'sharkia_fakous', 'Cairo, Egy

In [129]:
for word in query_cleaned['cleaned_geo'].str.split('_'):
    if word[0] in list(locations.values())[0]:
        query_cleaned['cleaned_geo'].replace(word, 'Saudi Arabia', inplace=True)
    elif word[0] in list(locations.values())[1]:
            query_cleaned['cleaned_geo'].replace(word, 'Kuwait', inplace=True)
    else:
        if word[0] in list(locations.values())[2]:
            query_cleaned['cleaned_geo'].replace(word, 'Egypt', inplace=True)
            

In [132]:
query_cleaned.sample(5)

,cleaned_text,cleaned_name,cleaned_geo
1549,هنفسي علي مهلنا بقي,mamhosni,الخُن
861,ايوة يا هندسة ما تسكتش,mamhosni,الخُن
2814,هههههههههههههههههههههههه,mamhosni,الخُن
2697,سلاماً على من سكن في القلب سلاماً على من تشتاق له الروح عهداً لك انت ال...,mamhosni,الخُن
1995,لازم إعدام الإخوان اللى فىالسجون لوقف نزيف الدم المصري,mamhosni,الخُن


### Pickle to conduct EDA in seperate notebook

In [133]:
pwd

'/home/jovyan/capstone-34/34.210.104.115/eg_twitter_raw/eg_timelines_raw'

In [134]:
cd

/home/jovyan


In [135]:
cd capstone-34/34.210.104.115/eg_twitter_pickled

/home/jovyan/capstone-34/34.210.104.115/eg_twitter_pickled


In [136]:
query_cleaned.to_pickle('../eg_twitter_pickled/user_timeline_mamhosni.p')